In [34]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten,Dense,Conv2D,MaxPool2D

import os

#### Data import and Preparation

In [22]:
train = pd.read_csv('./data/new_train.csv',index_col=0)
test = pd.read_csv('./data/new_test.csv',index_col=0)
train.head()

,dishes_zones,name_of_dish,file
0,Yoruba,amala,0_0_0008.jpeg
1,Yoruba,amala,0_0_0009.jpeg
2,Yoruba,amala,0_0_0010.jpeg
3,Yoruba,amala,0_0_0011.jpeg
4,Yoruba,amala,0_0_0012.jpeg


In [23]:
print('Train shape is {}'.format(train.shape))
print('Test shape is {}'.format(test.shape))

Train shape is (296, 3)
Test shape is (42, 3)


In [24]:
#transform data from image into dataframe

# np.ravel is equivalent to reshape(-1, order=order).

def change_image_format(data_row):
    number = data_row.shape[0]
    image = []  
    for i in range(number):
        img_address = './data/Yoruba/' + str(data_row.iloc[i])  
        img = np.ravel(mpimg.imread(img_address))   # Read an image from a file into an array.     
        image.append(img)
        
    df = pd.DataFrame(image)
    return df

In [26]:
#Get the Dataframe for the Train and Test file     
x_train = change_image_format(train['file'])
x_test = change_image_format(test['file'])

x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,71390,71391,71392,71393,71394,71395,71396,71397,71398,71399
0,201,192,175,203,194,177,196,187,172,207,...,113,147,128,113,147,128,113,147,128,113
1,177,179,174,177,179,174,178,180,175,179,...,227,225,236,232,216,227,223,212,223,219
2,245,247,244,245,247,244,245,247,246,246,...,253,248,252,253,248,252,253,248,252,253
3,193,189,180,190,186,177,189,186,177,186,...,210,215,215,213,217,217,215,219,219,217
4,103,105,84,107,109,87,106,105,84,111,...,141,155,140,133,156,140,127,154,138,122


In [31]:
le = LabelEncoder()  #instantiate the label encoder
 
y_train = le.fit_transform(train['name_of_dish'])

y_test = le.transform(test['name_of_dish'])

In [32]:
y_train[:5]

array([0, 0, 0, 0, 0])

#### Model Development

In [118]:
input_shape = (x_train.shape[1],)
model = Sequential()

model.add(Dense(12, input_shape=input_shape))  #first (input) Layer
model.add(Dense(24,activation='relu')) #first hidden layer
model.add(Dense(48,activation='relu')) #second hidden layer
model.add(Dense(96,activation='relu')) #fourth hidden layer

model.add(Dense(6,activation='softmax'))   #output 6 categories for each meal category
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train,y_train,epochs=43,verbose=False) #43 epochs fitted well.

In [117]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)


Test accuracy: 0.523809552192688


In [139]:
# save the model to disk
import pickle

filename = './models/nn_model.pkl'
pickle.dump(model, open(filename, 'wb'))

#### Result presentation and test

In [78]:
image_list = []
image_address = './data/Yoruba/0_0_0001.jpeg'
image_list.append(np.ravel(mpimg.imread(image_address)))

df = pd.DataFrame(image_list)
df

,0,1,2,3,4,5,6,7,8,9,...,71390,71391,71392,71393,71394,71395,71396,71397,71398,71399
0,178,179,163,178,180,166,183,189,175,186,...,255,187,223,255,190,227,255,190,227,254


In [103]:
model.predict(df)

array([[0., 0., 0., 0., 0., 1.]], dtype=float32)

In [110]:
arr = le.inverse_transform([np.argmax(model.predict(df))])

In [112]:
for x in arr:
    print('This is {}. Enjoy your meal'.format(x))

This is iyan. Enjoy your meal


In [138]:
# load the model from disk
filename = './models/nn_model.pkl'
loaded_model = pickle.load(open(filename, 'rb'))


image_list = []
image_address = 'efo.jpeg'

img = np.ravel(mpimg.imread(image_address))
image_list.append(img)
df_1 = pd.DataFrame(image_list)

#model prediction
loaded_model.predict(df_1)

array([[0., 0., 1., 0., 0., 0.]], dtype=float32)

In [128]:
pd.DataFrame(image_list.append(np.ravel(mpimg.imread(image_address))))

""


In [135]:
image_list

[[array([178, 179, 163, ..., 190, 227, 254], dtype=uint8)]]